In [1]:
import os
import tensorflow as tf
import numpy as np
import tvm
import tvm.relay as relay
import tvm.contrib.graph_runtime as runtime
from tvm.relay.expr_functor import ExprMutator
import riptide.models
from riptide.get_models import get_model
from riptide.binary.binary_layers import Config, DQuantize, XQuantize

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
config = Config(actQ=DQuantize, weightQ=XQuantize, bits=1, use_act=False, use_bn=False, use_maxpool=True)

In [4]:
with config:
    model = get_model('vggnet')
#model = riptide.models.vggnet_normal.vggnet()
#model = tf.keras.models.Sequential(model.layers[:30])
#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Conv2D(filters=96, strides=2, padding='same', kernel_size=7, use_bias=False, data_format='channels_last'))
#model.add(tf.keras.layers.BatchNormalization(center=False, scale=False))

In [5]:
test_input = tf.keras.Input(shape=[224, 224, 3], batch_size=1, dtype='float32')
output = model(test_input)

W0715 22:44:29.646168 139757493847872 deprecation.py:323] From /home/Riptide/riptide/binary/binary_layers.py:526: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
W0715 22:44:29.681277 139757493847872 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:279: calling Layer.add_update (from tensorflow.python.keras.engine.base_layer) with inputs is deprecated and will be removed in a future version.
Instructions for updating:
`inputs` is now automatically inferred


In [6]:
func, params = relay.frontend.from_keras(model, shape={'input_1': [1, 224, 224, 3]}, layout='NHWC')

In [7]:
print(func)

v0.0.1
def @main(%input_1: Tensor[(1, 224, 224, 3), float32], %v_param_1: Tensor[(3, 3, 96, 256), float32], %v_param_2: Tensor[(256,), float32], %v_param_3: Tensor[(256,), float32], %v_param_4: Tensor[(3, 3, 256, 256), float32], %v_param_5: Tensor[(256,), float32], %v_param_6: Tensor[(256,), float32], %v_param_7: Tensor[(3, 3, 256, 256), float32], %v_param_8: Tensor[(256,), float32], %v_param_9: Tensor[(256,), float32], %v_param_10: Tensor[(3, 3, 256, 512), float32], %v_param_11: Tensor[(512,), float32], %v_param_12: Tensor[(512,), float32], %v_param_13: Tensor[(3, 3, 512, 512), float32], %v_param_14: Tensor[(512,), float32], %v_param_15: Tensor[(512,), float32], %v_param_16: Tensor[(3, 3, 512, 512), float32], %v_param_17: Tensor[(512,), float32], %v_param_18: Tensor[(512,), float32], %v_param_19: Tensor[(3, 3, 512, 512), float32], %v_param_20: Tensor[(512,), float32], %v_param_21: Tensor[(512,), float32], %v_param_22: Tensor[(3, 3, 512, 512), float32], %v_param_23: Tensor[(512,), floa

In [7]:
with relay.build_config(opt_level=0):
    graph, lib, params = relay.build(func, target='llvm', params=params)
    #out = intrp.evaluate(func)(np.random.uniform(size=(1, 3, 28, 28)))

W0715 22:44:35.888565 139757493847872 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 4096, 'int16'), (1000, 1, 512, 'uint8'), 1, 1, 'uint8', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0715 22:44:35.894528 139757493847872 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 4096, 'int16'), (4096, 1, 512, 'uint8'), 1, 1, 'uint8', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0715 22:44:35.899775 139757493847872 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 100352, 'int16'), (4096, 1, 3136, 'uint8'), 1, 1, 'uint8', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0715 22:44:35.907642 139757493847872 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_conv2d_nhwc', (1, 28, 28, 512, 'int16'), (3, 3, 1, 64, 512

TVMError: Traceback (most recent call last):
  [bt] (8) /home/tvm/build/libtvm.so(tvm::relay::backend::GraphRuntimeCodegen::VisitExpr_(tvm::relay::CallNode const*)+0x60a) [0x7f1b41e7532a]
  [bt] (7) /home/tvm/build/libtvm.so(+0x74c48c) [0x7f1b41e5148c]
  [bt] (6) /home/tvm/build/libtvm.so(tvm::relay::CompileEngineImpl::LowerInternal(tvm::relay::CCacheKey const&)+0x43b) [0x7f1b41e5d54b]
  [bt] (5) /home/tvm/build/libtvm.so(tvm::relay::ScheduleGetter::Create(tvm::relay::Function const&)+0x6b2) [0x7f1b41e5b8a2]
  [bt] (4) /home/tvm/build/libtvm.so(tvm::relay::ScheduleGetter::VisitExpr(tvm::relay::Expr const&)+0xae) [0x7f1b41e5c3fe]
  [bt] (3) /home/tvm/build/libtvm.so(tvm::relay::ExprFunctor<tvm::Array<tvm::Tensor, void> (tvm::relay::Expr const&)>::VisitExpr(tvm::relay::Expr const&)+0xd2) [0x7f1b41e58f22]
  [bt] (2) /home/tvm/build/libtvm.so(std::_Function_handler<tvm::Array<tvm::Tensor, void> (tvm::NodeRef const&, tvm::relay::ExprFunctor<tvm::Array<tvm::Tensor, void> (tvm::relay::Expr co

In [ ]:
module = runtime.create(graph, lib, tvm.context('llvm', 0))
module.set_input(**params)

In [ ]:
module.set_input('input_1', np.random.uniform(size=(1, 224, 224, 3)))
module.run()
print(module.get_output(0).shape)

In [ ]:
# evaluate
ctx = tvm.cpu()
print("Evaluate inference time cost...")
ftimer = module.module.time_evaluator("run", ctx, number=1, repeat=10)
prof_res = np.array(ftimer().results) * 1000  # convert to millisecond
print("Mean inference time (std dev): %.2f ms (%.2f ms)" %
      (np.mean(prof_res), np.std(prof_res)))

In [ ]:
test_layer = tf.keras.layers.Conv2D(filters=10, kernel_size=3, data_format='channels_last')
test_input = tf.keras.Input(batch_size=1, shape=[32, 32, 32])

In [ ]:
test_output = test_layer(test_input)

In [ ]:
test_layer.get_weights()[0].shape

In [ ]:
len(test_layer.input_shape)